<!-- Banner Image -->
<img src="https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/landingpage/brevdevnotebooks.png" width="100%">

<!-- Links -->
<center>
  <a href="https://console.brev.dev" style="color: #06b6d4;">Console</a> •
  <a href="https://brev.dev" style="color: #06b6d4;">Docs</a> •
  <a href="/" style="color: #06b6d4;">Templates</a> •
  <a href="https://discord.gg/NVDyv7TUgJ" style="color: #06b6d4;">Discord</a>
</center>

# ⚡ Fine-Tuning Showdown: Unsloth vs HuggingFace

## Real Benchmarks. Same Model. Same Dataset. YOUR GPU.

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 30px; border-radius: 10px; color: white; margin-bottom: 20px;">
  <h2 style="margin-top: 0; color: white;">🎯 What You'll Discover</h2>
  <p style="font-size: 18px; line-height: 1.6;">
    <strong>Stop guessing. Start measuring.</strong><br/><br/>
    We'll train <strong>the same model</strong> with <strong>the same dataset</strong> using two frameworks:<br/>
    ⚡ <strong>Unsloth</strong> - Claims 2× speed, let's verify<br/>
    🤗 <strong>HuggingFace</strong> - Vanilla baseline (PEFT + Trainer)<br/>
    <br/>
    <strong>🔥 GPU starts training in 60 seconds. Side-by-side results in 10 minutes.</strong>
  </p>
</div>

## 📋 Prerequisites

- **GPU**: NVIDIA GPU with 16GB+ VRAM (A100, H100, L40S, RTX 4090, RTX 3090)
- **CUDA**: 11.8+ or 12.1+
- **Python**: 3.10+
- **Disk Space**: 20GB free (for models + datasets)

---

## 🎬 Quick Start: What Happens Next

The next few cells will:
1. **Install Unsloth** (~30 sec)
2. **Load Qwen 1.5B model** (~15 sec)
3. **Start training** immediately (2-3 min)
4. **Train with vanilla HF** (same config)
5. **Compare ALL metrics** side-by-side

**Total time: ~10 minutes for complete comparison** ⚡

---

#### 💬 Questions? Join us on [Discord](https://discord.gg/NVDyv7TUgJ) or reach out on [X/Twitter](https://x.com/brevdev)

**📝 Notebook Tips**: Press `Shift + Enter` to run cells. A `*` means running, a number means complete.

---